## Getting Geo-coordinates for WSJ Colleges
Here we are going to use a couple of Python tools to make a database of the Latitude / Longitude locations for the different schools contained in the report. I'm doing this to compare the speed and accuracy of the included Power BI ArcGIS maps with a hard-coding the coordinates. 

Our strategy is:
- Create a search string using the college name and city.
- Use the **[Geopy](https://github.com/geopy/geopy)** package to query Google Maps for each address.
- Map the addresses / coordinates to the existing database.
- Save the database as a new file.

First, we read in the WSJ data and create a search string.

In [14]:
import pandas as pd

wsj = pd.read_csv('wsj_data.csv')
wsj.head()

,rank,college,city_state,overall,outcome,resources,engagement,environment,right_choice,salary,default_rate,class,loc_string
0,1,Harvard University,"Cambridge, MA",91.9,39.5,29.8,15.6,7.0,9.09,91000,0.9,Private,"Harvard University, Cambridge, MA, USA"
1,2,Columbia University,"New York, NY",90.6,39.0,27.0,16.1,7.8,8.06,74000,1.4,Private,"Columbia University, New York, NY, USA"
2,3,Massachusetts Institute of Technology,"Cambridge, MA",90.4,38.2,29.2,15.8,7.2,9.11,90000,1.1,Private,"Massachusetts Institute of Technology, Cambrid..."
3,3,Stanford University,"Stanford, CA",90.4,38.9,26.2,17.4,7.9,8.96,83000,0.8,Private,"Stanford University, Stanford, CA, USA"
4,5,Duke University,"Durham, NC",90.2,39.5,26.7,17.2,6.8,9.19,77000,0.4,Private,"Duke University, Durham, NC, USA"


For each college, we're going to create a search string as if we were looking it up in Google Maps. It's important to include as much information as we have so that the location service doesn't get confused with institutions in other countries, for example.

In [16]:
if not 'loc_string' in wsj.columns:
    wsj['loc_string'] = wsj.apply(lambda s: '{}, {}, USA'.format(s.college, s.city_state), axis=1)
    wsj.to_csv('wsj_data.csv', encoding='utf-8', index=None)

print(wsj.loc_string[0:5])

0               Harvard University, Cambridge, MA, USA
1               Columbia University, New York, NY, USA
2    Massachusetts Institute of Technology, Cambrid...
3               Stanford University, Stanford, CA, USA
4                     Duke University, Durham, NC, USA
Name: loc_string, dtype: object


In [18]:
getCoords(wsj.loc_string[0])

Address      Cambridge, MA 02138, USA
Latitude                       42.377
Longitude                    -71.1167
Name: Harvard University, Cambridge, MA, USA, dtype: object

In [17]:
def getCoords(search_string):
    '''Takes a search term, queries Google and returns the geocoordinates.'''
    import requests
    
    try:
        query = search_string.replace(' ', '+')
        response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address={}'.format(query))
        response_from_google = response.json()
        
        address = response_from_google['results'][0]['formatted_address']
        latitude = response_from_google['results'][0]['geometry']['location']['lat']
        longitude = response_from_google['results'][0]['geometry']['location']['lng']
        
        return pd.Series(name=search_string, \
                         data={'Address': address, 'Latitude': latitude, 'Longitude': longitude})
    except:
        return pd.Series(name=search_string, data={'Address': None, 'Latitude': None, 'Longitude': None})

In [ ]:
getCoords(wsj.loc)

In [21]:
geodf = pd.DataFrame()

for ind, school in wsj.loc_string.iteritems():
    data = getCoords(school)
    geodf = geodf.append(data)
    print(school, '\n\t\t ', data)

Harvard University, Cambridge, MA, USA  -->  Address      Cambridge, MA 02138, USA
Latitude                       42.377
Longitude                    -71.1167
Name: Harvard University, Cambridge, MA, USA, dtype: object
Columbia University, New York, NY, USA  -->  Address      116th St & Broadway, New York, NY 10027, USA
Latitude                                          40.8075
Longitude                                        -73.9626
Name: Columbia University, New York, NY, USA, dtype: object
Massachusetts Institute of Technology, Cambridge, MA, USA  -->  Address      None
Latitude     None
Longitude    None
Name: Massachusetts Institute of Technology, Cambridge, MA, USA, dtype: object
Stanford University, Stanford, CA, USA  -->  Address      450 Serra Mall, Stanford, CA 94305, USA
Latitude                                     37.4275
Longitude                                    -122.17
Name: Stanford University, Stanford, CA, USA, dtype: object
Duke University, Durham, NC, USA  -->  Ad

In [22]:
geodf.to_csv('wsj_locs.csv', index=None)